In [8]:
import json

database = {}

with open("data/database.json", "r") as f:
    database = json.load(f)

db = database["db"]

In [9]:
from pathlib import Path 

datapath = Path(r"E:\Games\SteamLibrary\steamapps\common\Stationeers\Stationpedia\exported_textures")


In [10]:
import os

images = list(datapath.glob("*.png"))
names = [image.name  for image in images]


In [44]:
image_candidates = {}

def filter_candidates(candidates):
    max_match_len = 0
    filtered_matches = []

    # go for longest match
    for can in candidates:
        name, match, mapping = can
        match_len = len(match)
        if match_len > max_match_len:
            max_match_len = match_len
            filtered_matches = [(name, match, mapping)]
        elif match_len == max_match_len:
            filtered_matches.append((name, match, mapping))

    # choose better matches
    if len(filtered_matches) > 1:
        better_matches = []
        for can in filtered_matches:
            name, match, mapping = can
            if mapping.startswith("Item") and mapping in name:
                better_matches.append((name, match, mapping))
            elif mapping.startswith("Structure") and mapping in name:
                better_matches.append((name, match, mapping))
        if len(better_matches) > 0:
            filtered_matches = better_matches

    #exclude build states if we have non build states
    if len(filtered_matches) > 1:
        non_build_state = []
        for can in filtered_matches:
            name, match, mapping = can
            if "BuildState" not in name:
                non_build_state.append((name, match, mapping))
        if len(non_build_state) > 0:
            filtered_matches = non_build_state

    #prefer matches without extra tags
    if len(filtered_matches) > 1:
        direct = []
        for can in filtered_matches:
            name, match, mapping = can
            if f"{match}-" in name:
                direct.append((name, match, mapping))
        if len(direct) > 0:
            filtered_matches = direct
    
    #filter to unique filenames
    if len(filtered_matches) > 1:
        unique_names = []
        unique_matches = []
        for can in filtered_matches:
            name, match, mapping = can
            if name not in unique_names:
                unique_names.append(name)
                unique_matches.append((name, match, mapping))
        filtered_matches = unique_matches

    #prefer not worse matches
    if len(filtered_matches) > 1:
        not_worse = []
        for can in filtered_matches:
            name, match, mapping = can
            if name.startswith("Item") and not mapping.startswith("Item"):
                continue
            elif name.startswith("Structure") and not mapping.startswith("Structure"):
                continue
            elif name.startswith("Kit") and not mapping.startswith("Kit"):
                continue
            elif not name.startswith(match):
                continue
            not_worse.append((name, match, mapping))
        if len(not_worse) > 0:
            filtered_matches = not_worse

    #if we have colored variants take White
    if len(filtered_matches) > 1:
        for can in filtered_matches:
            name, match, mapping = can
            if f"_White" in name:
                return [name]

    return [name for name, _, _ in filtered_matches]

for entry in db.values():
    candidates = []
    for name in names:
        if entry["name"] in name:
            candidates.append((name, entry["name"], entry["name"]))
        if entry["name"].removeprefix("Item") in name:
            candidates.append((name, entry["name"].removeprefix("Item"), entry["name"]))
        if entry["name"].removeprefix("Structure") in name:
            candidates.append((name, entry["name"].removeprefix("Structure"), entry["name"]))
    image_candidates[entry["name"]] = filter_candidates(candidates)



In [46]:
# rematch items to super structure?
for name in image_candidates.keys():
    for other in image_candidates.keys():
        if name != other and name in other:
            if len(image_candidates[name]) > 0 and len(image_candidates[other]) == 0:
                image_candidates[other] = image_candidates[name]

In [47]:
to_copy = []
for name, candidates in image_candidates.items():
    if len(candidates) != 1:
        print(name, candidates)
        if len(candidates) > 1:
            #take first as fallback
            to_copy.append((name, candidates[0]))
    else:
        # print(name, candidates)
        to_copy.append((name, candidates[0]))


ItemBiomass []
StructureBlocker []
CartridgePlantAnalyser []
StructureElevatorLevelIndustrial []
ItemPlantEndothermic_Creative []
Flag_ODA_10m []
Flag_ODA_4m []
Flag_ODA_6m []
Flag_ODA_8m []
ItemHorticultureBelt []
ItemKitLiquidRegulator []
ItemKitPortablesConnector []
Landingpad_GasConnectorInwardPiece []
Landingpad_LiquidConnectorInwardPiece []
ItemMushroom ['ItemMushroom-resources.assets-3022.png', 'ItemMushroom-resources.assets-9304.png']
StructurePlinth []
ItemPlantThermogenic_Creative []


In [48]:
import shutil
from alive_progress import alive_bar
destpath = Path("img/stationpedia")
total_files = len(to_copy)

with alive_bar(total_files) as bar:
    for name, file in to_copy:
        source = datapath / file
        dest = destpath / f"{name}.png"
        shutil.copy(source, dest)
        bar()


|████████████████████████████████████████| 1223/1223 [100%] in 0.8s (1494.22/s) 
